In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### 보스턴 주택 가격 데이터 세트를 Peceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
 * 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [4]:
from sklearn.datasets import load_boston

boston = load_boston()
print(boston)

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]]), 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 1

In [5]:
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target #target값
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Weight와 Bias의 Update 값을 계산하는 함수 생성.
* w1은 RM(방의 계수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* bias는 Bias
* N은 입력 데이터 건수
![](https://raw.githubusercontent.com/chulminkw/CNN_PG/main/utils/images/Weight_update.png)


In [6]:
# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수. 
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨. 
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 loss로 반환.
'''
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    # 데이터 건수
    N = len(target)
    # 예측 값. 
    predicted = w1 * rm + w2*lstat + bias
    # 실제값과 예측값의 차이 
    diff = target - predicted
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산. (위 내용 참고, 미분 취하는 계산임)
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
    # Mean Squared Error값을 계산. 
    mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환. 
    return bias_update, w1_update, w2_update, mse_loss
'''
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate):
    # rm, lstat의 영향이 가장 커서 이 두 개로만 연습할 것임
    # w1, w2가 들어오는 이유는 예측값 계산하기 위함
    
    N = len(target) # def의 인자로 들어오는 건 전부 array(배열) => sum 속도빠름. loop돌리지 않고 numpy 써서 더 좋음.
    # 주택가격 전체 건수
    
    predicted = w1*rm + w2*lstat + bias
    # 예측값i
    
    diff = target-predicted # array 인수 506개를 한번에 계산
    # 차이i = 실제값i-예측값i
    
    w1_update = -learning_rate*(2/N)*np.dot(rm.T, diff) # np.dot()위해 앞에있는 rm 전치행렬로 바꿈
    w2_update = -learning_rate*(2/N)*np.dot(lstat.T, diff)
    # η(dLoss(W)/dw1)
    # η = learning rate = step
    
    bias_update1 = -learning_rate*(2/N)*np.sum(diff)
    bias_factors = np.ones((N,))
    bias_update = -learning_rate*(2/N)*np.dot(bias_factors.T, diff)
    bias_update1
    bias_update
    
    mse_loss = np.mean(np.square(diff)) # 차이의 제곱의 평균
    
    return bias_update, w1_update, w2_update, mse_loss # 업데이트할 bias와 weight, mse 리턴

### Gradient Descent 를 적용하는 함수 생성
* iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용. 

In [ ]:
np.zeros((3,)), np.ones((2,))

In [7]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0] # 현재 features 안에는 bostonDF에서 rm, lstat 두 피처만 뽑혀있었음
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행 (get_update_weights_value 함수) 
    for i in range(iter_epochs):
        # weight/bias update 값 계산 
        bias_update, w1_update, w2_update, loss = get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate)
        # weight/bias의 update 적용 (이전 셀 참고)
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', loss) #loss값이 계속 줄어드나 확인용
        
    return w1, w2, bias

### Gradient Descent 적용
* 신경망은 데이터를 정규화/표준화 작업을 미리 선행해 주어야 함. 
* 이를 위해 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # default값 안 주면 0~1 사이 값으로 바꿔줌
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']]) #bostonDF에서 rm, lstat 2개의 피처만 가져올것임
print(scaled_features[:15])
print(bostonDF['PRICE'])
print(bostonDF['PRICE'].values)

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() # default값 안 주면 0~1 사이 값으로 바꿔줌
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']]) #bostonDF에서 rm, lstat 2개의 피처만 가져올것임

w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
# bostonDF['PRICE'].values를 해줘서 value값만 들어있는 array배열이 됨
# iter_epochs=1000 => 1000번돌리는것. 
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)
'''
최초 w1, w2, bias: [0.] [0.] [1.]
Epoch: 1 / 1000
w1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261
Epoch: 2 / 1000
w1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195
Epoch: 3 / 1000
w1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575
~~~
Epoch: 1000 / 1000
w1: [18.54730416] w2: [-13.01074165] bias: [16.77763613] loss: 38.46753354662152
##### 최종 w1, w2, bias #######
[18.54730416] [-13.01074165] [16.77763613]
'''

최초 w1, w2, bias: [0.] [0.] [1.]
Epoch: 1 / 1000
w1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261
Epoch: 2 / 1000
w1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195
Epoch: 3 / 1000
w1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575
Epoch: 4 / 1000
w1: [0.93124998] w2: [0.39159032] bias: [2.65334123] loss: 476.7031232605375
Epoch: 5 / 1000
w1: [1.14941104] w2: [0.48110116] bias: [3.03885015] loss: 455.41666565492966
Epoch: 6 / 1000
w1: [1.36202867] w2: [0.56744065] bias: [3.41383227] loss: 435.2718794853261
Epoch: 7 / 1000
w1: [1.56925388] w2: [0.65069613] bias: [3.7785751] loss: 416.20726135905875
Epoch: 8 / 1000
w1: [1.77123356] w2: [0.73095251] bias: [4.13335831] loss: 398.1646216743121
Epoch: 9 / 1000
w1: [1.96811059] w2: [0.8082924] bias: [4.47845392] loss: 381.0889060727257
Epoch: 10 / 1000
w1: [2.16002396] w2: [0.88279616] bias: [4.81412652] loss: 364.9280265121499
Epoch: 11 / 1000
w1: [2.34710885] w2:

'\n최초 w1, w2, bias: [0.] [0.] [1.]\nEpoch: 1 / 1000\nw1: [0.24193162] w2: [0.10311943] bias: [1.43065613] loss: 548.0813043478261\nEpoch: 2 / 1000\nw1: [0.47767212] w2: [0.20269304] bias: [1.84955238] loss: 522.964778344195\nEpoch: 3 / 1000\nw1: [0.70739021] w2: [0.29881838] bias: [2.25700994] loss: 499.19625820107575\n~~~\nEpoch: 1000 / 1000\nw1: [18.54730416] w2: [-13.01074165] bias: [16.77763613] loss: 38.46753354662152\n##### 최종 w1, w2, bias #######\n[18.54730416] [-13.01074165] [16.77763613]\n'

### 계산된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산. 

In [9]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,26.322000
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,24.281207
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,28.830886
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,28.557652
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,28.229120
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,25.724125
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.646539
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,19.802559
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,14.009758
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,19.941503


### Keras를 이용하여 보스턴 주택가격 모델 학습 및 예측
* Dense Layer를 이용하여 퍼셉트론 구현. units는 1로 설정. 

In [10]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])
model.fit(scaled_features, bostonDF['PRICE'].values, epochs=1000)


2021-12-27 03:00:41.201542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-27 03:00:41.319708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-27 03:00:41.320843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


KeyboardInterrupt: 

### Keras로 학습된 모델을 이용하여 주택 가격 예측 수행. 

In [ ]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE'] = predicted
bostonDF.head(10)


### Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 한건만 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 Batch 건수만큼 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
print(bostonDF.shape)
bostonDF.head()

### SGD 기반으로 Weight/Bias update 값 구하기

In [ ]:
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    
    # 데이터 건수
    N = target_sgd.shape[0]
    # 예측 값. 
    predicted_sgd = w1 * rm_sgd + w2*lstat_sgd + bias
    # 실제값과 예측값의 차이 
    diff_sgd = target_sgd - predicted_sgd
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_sgd.T, diff_sgd))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_sgd.T, diff_sgd))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_sgd))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

### SGD 수행하기

In [ ]:
print(bostonDF['PRICE'].values.shape)
print(np.random.choice(bostonDF['PRICE'].values.shape[0], 1))
print(np.random.choice(506, 1))

In [ ]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용. 
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # iteration 시마다 stochastic gradient descent 를 수행할 데이터를 한개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택. 
        stochastic_index = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]
        # SGD 기반으로 Weight/Bias의 Update를 구함.  
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        
        # SGD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_SGD'] = predicted
bostonDF.head(10)

### iteration시마다 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [ ]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    # 데이터 건수
    N = target_batch.shape[0]
    # 예측 값. 
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이 
    diff_batch = target_batch - predicted_batch
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

In [ ]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes)

bostonDF['RM'].values[batch_indexes]



In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 갯수만큼 데이터를 임의로 선택. 
        batch_indexes = np.random.choice(target.shape[0], batch_size)
        rm_batch = rm[batch_indexes]
        lstat_batch = lstat[batch_indexes]
        target_batch = target[batch_indexes]
        # Batch GD 기반으로 Weight/Bias의 Update를 구함. 
        bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
        
        # Batch GD로 구한 weight/bias의 update 적용. 
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 함.
            predicted = w1 * rm + w2*lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH_RANDOM'] = predicted
bostonDF.head(10)

### iteration 시에 순차적으로 일정한 batch 크기만큼의 데이터를 전체 학습데이터에 걸쳐서 가져오는 Mini-Batch GD 수행

In [ ]:
for batch_step in range(0, 506, 30):
    print(batch_step)

In [ ]:
bostonDF['PRICE'].values[480:510]

In [ ]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음. 
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정. 
    np.random.seed = 2021
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.zeros((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)
    
    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]
    
    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행. 
    for i in range(iter_epochs):
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size):
            # batch_size만큼 순차적인 데이터를 가져옴. 
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]
        
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

            # Batch GD로 구한 weight/bias의 update 적용. 
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
        
            if verbose:
                print('Epoch:', i+1,'/', iter_epochs, 'batch step:', batch_step)
                # Loss는 전체 학습 데이터 기반으로 구해야 함.
                predicted = w1 * rm + w2*lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)
        
    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=5000, batch_size=30, verbose=True)
print('##### 최종 w1, w2, bias #######')
print(w1, w2, bias)

In [ ]:
predicted = scaled_features[:, 0]*w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)

### Mini BATCH GD를 Keras로 수행
* Keras는 기본적으로 Mini Batch GD를 수행

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당. 
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=30, epochs=1000)

In [ ]:
predicted = model.predict(scaled_features)
bostonDF['KERAS_PREDICTED_PRICE_BATCH'] = predicted
bostonDF.head(10)